# Live Sandbox

In [1]:
# IMPORT LIBRARIES
import pandas as pd
import math
from scipy import interpolate

In [2]:
# DEFINE CONSTANTS & VARIABLES

## Some common physical constants, mainly used to calculate maxFlux (the upper bound of energy flux received by a planet...
## ...that would allow it to be habitable)
BIGG       = 6.67428e-11       # gravitational constant
PI         = 3.1415926535
A          = 0.7344            # Pierrehumbert's Constant
SB         = 5.670373e-8       # Stefan-Boltzmann Constant
LH2O       = 2.425e6           # Latent Heat Capacity of Water
RGAS       = 461.5             # Universal Gas Constant
PLINE      = 1e4               
PREF       = 610.616           # Reference Pressure
TREF       = 273.13            # Reference Temperature
K0         = 0.055             # A constant in Runaway Greenhouse calculation

## Albedo value boundaries
ALBMINELSE = 0.05              # General lower bound
ALBMAXELSE = 0.8               # General upper bound
ALBMING    = 0.25              # Lower bound for planets orbiting G-type stars
ALBMAXM    = 0.35              # Upper bound for planets orbiting M-type stars

## The counterpart to maxFlux
MINFLUX    = 67                

## Definitions of units of measurement, mainly used to convert exoplanet.org data into SI units
MEARTH     = 5.972186e24       # Earth mass in kilograms
REARTH     = 6378100           # Earth's radius in meters
S0         = 1362              # Solar constant in watts per square meter
MSUN       = 1.988416e30       # Solar mass in kilograms
RSUN       = 6.957e8           # Solar radius in meters
LSUN       = 3.828e26          # Solar luminosity in watts
RJUP       = 7.1492e7          # Jovian radius in meters
MJUP       = 1.8982e27         # Jovian mass in kilograms
AU         = 1.496e11          # The astronomical unit in meters

In [3]:
# DEFINE FUNCTIONS

## Probability distribution of eccentricity (uses eccentricity data (and their measurement uncertainties) from exoplanet.org)
def pofe(ecc,mu,sigma):
    return ((sigma*math.sqrt(2*math.pi))**(-1))*math.exp(-(((ecc-mu)**2)/(2*sigma**2)))/1000

## A function to calculate probability of planet's rocky-ness/terrestriality
def fpRocky (mPlanet,rPlanet,exoName):
    mPlanet = mPlanet/MEARTH # Convert the unit to Earth's masses (the Zeng-Sasselov dataset table uses Earth's mass and radius as the units)
    #print("Mass in Earth's masses:", mPlanet)
    rPlanet = rPlanet/REARTH # Convert the unit to Earth's radii
    
    # Calculate mu1    
    #print("mu1")
    mu1 = 0.0       
    mZSimin1 = 0    
    rZSimin1 = 0
    
    # this conditional block tries to find the appropriate position for mPlanet within the list of 'pure MgSiO3' mass
    for row in rowNum:   # rowNum contains row numbers of the ZS table
        mZSi = zs.loc[row, "M-PureMgSiO3"] # load a mass value from a row in the ZS table
        rZSi = zs.loc[row, "R-PureMgSiO3"] # load the corresponding 'pure MgSiO3' radius from the same row
        #print(row, " massIter:", mZSi)
        # this block compares mPlanet to the currently loaded mass value from the ZS table
        if mPlanet == mZSi: 
            mu1 = rZSi
            break
        elif mPlanet > mZSi:
            mZSimin1 = mZSi
            rZSimin1 = rZSi
        else:
            f = interpolate.interp1d(zs.loc[(row-1):(row), "M-PureMgSiO3"], zs.loc[(row-1):(row), "R-PureMgSiO3"], kind='linear', assume_sorted=True)
            mu1 = f(mPlanet)
            print(zs.loc[(row-1):(row), "M-PureMgSiO3"])
            break 

    ### Calculate mu2, the upper bound of the 'not-zero chance' range
    #print("mu2")
    mu2 = 0.0
    mZSimin1 = 0
    rZSimin1 = 0
    for row in rowNum:
        mZSi = zs.loc[row, "M-MgSiO3-H2O-5050"]
        rZSi = zs.loc[row, "R-MgSiO3-H2O-5050"]
        #print(row, " massIter:", mZSi)
        if mPlanet == mZSi:
            mu2 = rZSi
            break
        elif mPlanet > mZSi:
            mZSimin1 = mZSi
            rZSimin1 = rZSi
        else:
            f = interpolate.interp1d(zs.loc[(row-1):(row), "M-MgSiO3-H2O-5050"], zs.loc[(row-1):(row), "R-MgSiO3-H2O-5050"], kind='linear', assume_sorted=True)
            mu2 = f(mPlanet)
            print(zs.loc[(row-1):(row), "M-MgSiO3-H2O-5050"], "\n")
            break

    ### Calculate sigma1
    sigma1 = (mu2-mu1)/3
    #print("sigma: ",sigma1)
    
    pRocky = 0
    if rPlanet <= mu1:
        pRocky = 1
    elif rPlanet >= mu2:
        pRocky = 0
    else: # use the T_M_p function from SEPHI
        pRocky = math.exp(-(0.5)*((rPlanet-mu1)/sigma1)**2)
        
    return pRocky

In [4]:
### Import exoplanet data from a CSV into a pandas dataframe
## TO DO redo exoplanetsInUse
exo = pd.read_csv (r'exoplanetsInUse_noKOI1.csv', low_memory=False)

### Set the column with the header NAME to be used as an index to identify row 
exo = exo.set_index("NAME", drop = False)

### Extract names of planets as a list (to be used as a calling list)
exoList = pd.DataFrame(exo, columns=['NAME'])
exoList = exoList['NAME'].values.tolist()

In [5]:
### Import CSV of Zeng & Sasselov boundaries
zs = pd.read_csv (r'zeng-sasselov_boundaries.csv')

### Set index using the RowNum column
zs = zs.set_index("RowNum", drop = False)

### Extract the column "RowNum" as a list (to be used as a calling list)
rowNum = pd.DataFrame(zs, columns=['RowNum'])
rowNum = rowNum['RowNum'].values.tolist()

In [6]:
## Subroutine to determine Habitability Index value

habIndex = []
habIndexWithName = []

for exoName in exoList:
    print("Name:", exoName)
    #Extract data of individual planets
    
    #### HOST STAR PROPERTIES
    ### Stellar radius (in solar radii)
    rStar = exo.loc[exoName, "RSTAR"]
    # Convert to SI
    rStar = rStar*RSUN
    ### Stellar temperature (in Kelvin)
    teffStar = exo.loc[exoName, "TEFF"]
    ### Stellar luminosity
    luminosity = 4*math.pi*rStar*rStar*SB*teffStar**4
    
    ###### PLANET PROPERTIES   
    ### Planetary radius (in Jovian radii)
    rPlanet = exo.loc[exoName, "R"]
    # If Rp is not available, calculate it from  transit depth
    if math.isnan(rPlanet) == 1:
        depth = exo.loc[exoName, "DEPTH"]
        rPlanet = math.sqrt(depth)*rStar
    # Convert to SI
    rPlanet = rPlanet*RJUP
    ### Planetary mass (in Jovian masses)
    mPlanet = exo.loc[exoName, "MASS"] 
    # If Mp is not available, calculate it from a common scaling law, [...]
    # using Rp
    if math.isnan(mPlanet) == 1:
        if rPlanet/REARTH <= 1:
            mPlanet = ((rPlanet/REARTH)**3.268)*MEARTH
        elif rPlanet/REARTH > 1:
            mPlanet = ((rPlanet/REARTH)**3.65)*MEARTH
    # Convert to SI
    mPlanet = mPlanet*MJUP
    ### Surface planet gravity (in SI)
    surfGrav = BIGG*mPlanet/(rPlanet**2)    
    
    
    
    ###### Orbital properties
    ### Orbital eccentricity
    ecc = exo.loc[exoName, "ECC"]
    ### Measurement uncertainty of orbital eccentricity    
    # Upper bound (relative from E)
    eccUpRel = exo.loc[exoName, "ECCUPPER"]
    # If measurement uncertainty is not available, assign it as 0.01
    if math.isnan(eccUpRel) == 1:
        eccUpRel = 0.01
    # Upper bound (relative from E)
    eccUpper = ecc + eccUpRel
    # Lower bound (relative from E)
    eccLowRel = exo.loc[exoName, "ECCLOWER"]
    # If measurement uncertainty is not available, assign it as 0.01
    if math.isnan(eccLowRel) == 1:
        eccLowRel = 0.01
    # Lower bound (absolute)
    eccLower = ecc - eccLowRel
    ### Orbital semi-major axis (in AU)
    semiAxis = exo.loc[exoName, "A"]      
    # Convert to SI
    semiAxis = semiAxis*AU
    
    
    ###### Calculate the upper and lower bounds of F_OLR [...]
    ###### that would allow for surface liquid water to exist
    ### lupa apa
    pStar = PREF*math.exp(LH2O/(RGAS*TREF))
    ### Upper bound: maximum F_OLR
    maxFlux = A*SB*(LH2O/(RGAS*math.log(pStar*math.sqrt(K0/(2*PLINE*surfGrav)))))**4
    ### Lower bound: minimum F_OLR is the constant MINFLUX

    
    ###### Probability of rocky-ness (new)
    pRocky = fpRocky(mPlanet,rPlanet,exoName)
        
    
    ###### Albedo (new)
    ### Boundaries
    albMin = ALBMINELSE
    albMax = ALBMAXELSE
    
    # Special conditions
    # For planets with M-type host star
    if teffStar >= 2300 and teffStar <=3800:
        albMax = ALBMAXM
    # For planets with G-type host star
    elif teffStar >= 5370 and teffStar <=5980:
        albMin = ALBMING
        
        
    
    ###### Calculate F_OLR
    ### Albedo increments
    da = 0.01
    ### Eccentricity increments
    de = 0.01
    ### Sum of pofe (probability of eccentricity);
    ### is used to normalize the index value, later)
    pofeSum = 0
    ### How many instances of F_OLR meets the requirements for [...]
    ### the planet to have surface liquid water? Each instances would be [...]
    ### multiplied by the probability of that value of F_OLR from occuring
    habFact = 0
    ### Incoming stellar radiation (instellation)
    flux0 = luminosity/(16*math.pi*semiAxis*semiAxis)

    ### Calculate H
    a = albMin
    while a < albMax:
        e = eccLower
        while e < eccUpper:
            flux = flux0*(1-a)/math.sqrt(1-e*e)
            pofeSum = pofeSum + pofe(e, ecc, eccUpRel)
            if flux < maxFlux and flux > MINFLUX:
                habFact = habFact + pofe(e, ecc, eccUpRel)
            e = e + de
        a = a + da   
    
    if ecc > 0.8:
        H = 0.0
    elif pofeSum != 0:
        H = (habFact/pofeSum)*pRocky
    else: # for error case
        H = 0.0
    
    habIndex.append(H)
    habIndexWithName.extend([exoName, ",", H])
    
        
    # print("Hab index:", H, "\n")    
    

Name: 51 Eri b
Name: 55 Cnc e
RowNum
26    7.714
27    9.423
Name: M-PureMgSiO3, dtype: float64
RowNum
25    7.392
26    9.043
Name: M-MgSiO3-H2O-5050, dtype: float64 

Name: BD +20 594 b
RowNum
29    13.87
30    16.73
Name: M-PureMgSiO3, dtype: float64
RowNum
29    16.18
30    19.48
Name: M-MgSiO3-H2O-5050, dtype: float64 

Name: CoRoT-1 b
Name: CoRoT-10 b
Name: CoRoT-11 b
Name: CoRoT-12 b
Name: CoRoT-13 b
Name: CoRoT-14 b
Name: CoRoT-16 b
RowNum
45    158.2
46    177.8
Name: M-PureMgSiO3, dtype: float64
RowNum
44    160.6
45    181.0
Name: M-MgSiO3-H2O-5050, dtype: float64 

Name: CoRoT-17 b
Name: CoRoT-18 b
Name: CoRoT-19 b
Name: CoRoT-2 b
Name: CoRoT-20 b
Name: CoRoT-22 b
RowNum
28    11.47
29    13.87
Name: M-PureMgSiO3, dtype: float64
RowNum
27    11.03
28    13.40
Name: M-MgSiO3-H2O-5050, dtype: float64 

Name: CoRoT-23 b
Name: CoRoT-24 b
Name: CoRoT-24 c
RowNum
32    24.07
33    28.68
Name: M-PureMgSiO3, dtype: float64
RowNum
31    23.36
32    27.94
Name: M-MgSiO3-H2O-5050, dty

C:\Users\Zulfa\AppData\Local\Temp\ipykernel_5708\3861487649.py:74: RuntimeWarning: divide by zero encountered in double_scalars
  maxFlux = A*SB*(LH2O/(RGAS*math.log(pStar*math.sqrt(K0/(2*PLINE*surfGrav)))))**4


RowNum
48    254.2
49    283.3
Name: M-MgSiO3-H2O-5050, dtype: float64 

Name: CoRoT-3 b
Name: CoRoT-4 b
RowNum
48    222.5
49    248.1
Name: M-PureMgSiO3, dtype: float64
RowNum
47    227.7
48    254.2
Name: M-MgSiO3-H2O-5050, dtype: float64 

Name: CoRoT-5 b
RowNum
44    140.3
45    158.2
Name: M-PureMgSiO3, dtype: float64
RowNum
43    142.0
44    160.6
Name: M-MgSiO3-H2O-5050, dtype: float64 

Name: CoRoT-6 b
Name: CoRoT-7 b
RowNum
23    4.043
24    5.077
Name: M-PureMgSiO3, dtype: float64
RowNum
22    3.780
23    4.763
Name: M-MgSiO3-H2O-5050, dtype: float64 

Name: CoRoT-8 b
RowNum
38    63.08
39    72.75
Name: M-PureMgSiO3, dtype: float64
RowNum
37    62.77
38    72.58
Name: M-MgSiO3-H2O-5050, dtype: float64 

Name: CoRoT-9 b
RowNum
48    254.2
49    283.3
Name: M-MgSiO3-H2O-5050, dtype: float64 

Name: EPIC 211822797 b
RowNum
24    5.077
25    6.297
Name: M-PureMgSiO3, dtype: float64
RowNum
23    4.763
24    5.972
Name: M-MgSiO3-H2O-5050, dtype: float64 

Name: EPIC 211945201 b
R

C:\Users\Zulfa\AppData\Local\Temp\ipykernel_5708\3861487649.py:110: RuntimeWarning: divide by zero encountered in double_scalars
  flux0 = luminosity/(16*math.pi*semiAxis*semiAxis)


RowNum
22    3.780
23    4.763
Name: M-MgSiO3-H2O-5050, dtype: float64 

Name: Kepler-1336 c
RowNum
22    3.203
23    4.043
Name: M-PureMgSiO3, dtype: float64
RowNum
22    3.780
23    4.763
Name: M-MgSiO3-H2O-5050, dtype: float64 

Name: Kepler-1337 b
RowNum
24    5.077
25    6.297
Name: M-PureMgSiO3, dtype: float64
RowNum
23    4.763
24    5.972
Name: M-MgSiO3-H2O-5050, dtype: float64 

Name: Kepler-1338 b
RowNum
15    0.5420
16    0.7143
Name: M-PureMgSiO3, dtype: float64
RowNum
15    0.6607
16    0.8653
Name: M-MgSiO3-H2O-5050, dtype: float64 

Name: Kepler-1339 b
RowNum
12    0.2255
13    0.3042
Name: M-PureMgSiO3, dtype: float64
RowNum
11    0.2074
12    0.2799
Name: M-MgSiO3-H2O-5050, dtype: float64 

Name: Kepler-134 b
RowNum
23    4.043
24    5.077
Name: M-PureMgSiO3, dtype: float64
RowNum
22    3.780
23    4.763
Name: M-MgSiO3-H2O-5050, dtype: float64 

Name: Kepler-134 c
RowNum
20    1.981
21    2.525
Name: M-PureMgSiO3, dtype: float64
RowNum
19    1.842
20    2.351
Name: M-M

In [7]:
# print(sum(habIndexList)/len(habIndexList)) #0.002815606263929704 #0.002844687739372335 (interpolate, iloc)
# print(habIndexList)
print(sum(habIndex)/len(habIndex))

0.0028039378224398435


In [8]:
# this doesn't append anything between 0 and 1??????
#with open('out_interpolate.txt','w') as f:
#    i = 1
#    for a in habIndexWithName:
#        if i == 3:
#            print(a, file=f)
#            i = 1
#        else:
#            print(a, file=f, end="")
#            i += 1

In [9]:
with open('out_interpolate_loc.txt','w') as f:
    i = 1
    for a in habIndex:
        print(a, file=f)

In [10]:
#print(0.48 * MJUP/MEARTH, ",", 1.2 * RJUP/REARTH)

In [11]:
#print(fpRocky (152.56323229048795,13.450776877126417,"WASP-96 b"))